# Introducción a la generación de descripciones de imágenes con IA

Propósito general:

Este cuaderno tiene como objetivo introducir a personas sin experiencia previa en el uso de la IA para la generación automática de descripciones de imágenes. A través de un ejemplo práctico, aprenderás a utilizar la biblioteca transformers de Hugging Face para cargar un modelo pre-entrenado, procesar una imagen y generar una descripción textual de la misma.

En este cuaderno, se abordarán los siguientes temas:

* Instalación de la biblioteca transformers: Se explicará cómo instalar la biblioteca transformers y sus dependencias necesarias en Google Colab.

* Carga del modelo y procesador: Aprenderás a cargar un modelo pre-entrenado de Hugging Face, específicamente el modelo "Salesforce/blip-image-captioning-base", junto con su procesador correspondiente.

* Carga y preparación de la imagen: Se te guiará en el proceso de cargar una imagen en el cuaderno y prepararla para ser procesada por el modelo.

* Generación de la descripción: Verás cómo utilizar el modelo para generar una descripción textual de la imagen cargada.

* Visualización de la descripción: Finalmente, se mostrará cómo imprimir la descripción generada por el modelo.


*   Elemento de lista



In [1]:
# Install the transformers library
!pip install transformers Pillow torch torchvision torchaudio -q
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 31.1 MB/s eta 0:00:00


In [2]:
# Initialize the processor and model from Hugging Face
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [ ]:
# Load an image
image = Image.open("/content/istockphoto-1403073162-612x612.jpg")

In [ ]:
# Prepare the image
inputs = processor(image, return_tensors="pt")

In [ ]:
# Generate captions
outputs = model.generate(**inputs)
caption = processor.decode(outputs[0],skip_special_tokens=True)

In [ ]:
print("Generated Caption:", caption)

## Interfaz interactiva con Gradio

En esta sección, se implementa una interfaz de usuario interactiva utilizando la biblioteca Gradio. Esta interfaz permite a los usuarios cargar sus propias imágenes y obtener descripciones generadas por el modelo BLIP de manera sencilla e intuitiva, sin necesidad de escribir código.

In [7]:
!pip install gradio -q

In [8]:
import gradio as gr

In [9]:
def generar_descripcion(imagen):
    # Ahora usando directamente el objeto PIL Image
    entradas = processor(images=imagen, return_tensors="pt")
    salidas = model.generate(**entradas)
    descripcion = processor.decode(salidas[0], skip_special_tokens=True)
    return descripcion

In [10]:
def descripcion_imagen(imagen):
    """
    Recibe una imagen PIL y devuelve una descripción.
    """
    try:
        descripcion = generar_descripcion(imagen)
        return descripcion
    except Exception as e:
        return f"Ocurrió un error: {str(e)}"

In [11]:
interfaz = gr.Interface(
    fn=descripcion_imagen,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Generación de descripciones de imágenes con BLIP",
    description="Sube una imagen para generar una descripción."
)
interfaz.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://12f953472d70255e56.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
